In [1]:
pip install nbconvert


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

def monthlist_for_year(y):
    return [f"{m:02d}" for m in range(1, 13)]

def build_plan_month(y):
    print("build_plan_month CALLED")
    return pd.DataFrame({"test": [1, 2, 3]})


In [11]:
print("build_plan_month exists:", "build_plan_month" in globals())


build_plan_month exists: True


In [12]:
import numpy as np
import pandas as pd
from pathlib import Path

# ---------------- Paths ----------------
YEAR = 2018
ENROLL_DIR = Path("~/econ470/a0/work/ma-data/ma/enrollment/Extracted Data").expanduser()
SA_DIR     = Path("~/econ470/a0/work/ma-data/ma/service-area/Extracted Data").expanduser()
OUT_DIR    = Path("~/econ470/a0/output").expanduser()
OUT_DIR.mkdir(parents=True, exist_ok=True)

def monthlist_for_year(y: int) -> list[str]:
    return [f"{m:02d}" for m in range(1, 13)]

# ---------------- Column schemas ----------------
CONTRACT_COLUMNS = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
]
ENROLL_COLUMNS = ["contractid", "planid", "ssa", "fips", "state", "county", "enrollment"]

# ---------------- Readers (encoding-safe) ----------------
def read_contract(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, skiprows=1, header=None,
                     encoding="latin1", encoding_errors="replace", low_memory=False)
    df = df.iloc[:, :len(CONTRACT_COLUMNS)]
    df.columns = CONTRACT_COLUMNS
    return df

def read_enroll(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, skiprows=1, header=None, na_values=["*"],
                     encoding="latin1", encoding_errors="replace", low_memory=False)
    df = df.iloc[:, :len(ENROLL_COLUMNS)]
    df.columns = ENROLL_COLUMNS
    return df

def read_service_area(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype="string",
                     encoding="latin1", encoding_errors="replace", low_memory=False)
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    if "contract_id" in df.columns:
        df = df.rename(columns={"contract_id": "contractid"})
    if "ssa" in df.columns:
        df["ssa"] = pd.to_numeric(df["ssa"], errors="coerce")
    if "fips" in df.columns:
        df["fips"] = pd.to_numeric(df["fips"], errors="coerce")

    keep = [c for c in ["contractid", "ssa", "fips", "state", "county"] if c in df.columns]
    return df[keep].copy()

# ---------------- Monthly loader ----------------
def load_month(m: str, y: int) -> pd.DataFrame:
    c_path = ENROLL_DIR / f"CPSC_Contract_Info_{y}_{m}.csv"
    e_path = ENROLL_DIR / f"CPSC_Enrollment_Info_{y}_{m}.csv"

    if not c_path.exists():
        raise FileNotFoundError(f"Missing contract file: {c_path}")
    if not e_path.exists():
        raise FileNotFoundError(f"Missing enrollment file: {e_path}")

    contract = read_contract(c_path).drop_duplicates(subset=["contractid", "planid"])
    enroll   = read_enroll(e_path)

    return (
        contract
        .merge(enroll, on=["contractid", "planid"], how="left")
        .assign(month=int(m), year=y)
    )

# ---------------- Build monthly panel ----------------
def build_plan_month(y: int) -> pd.DataFrame:
    months = monthlist_for_year(y)
    plan_month = pd.concat([load_month(m, y) for m in months], ignore_index=True)

    # key used for service-area merge: prefer fips, else ssa
    plan_month["county_key"] = plan_month["fips"].where(plan_month["fips"].notna(), plan_month["ssa"])
    return plan_month

# ---------------- Service area keyed ----------------
def build_service_area_keyed(y: int) -> pd.DataFrame:
    months = monthlist_for_year(y)
    files = [SA_DIR / f"MA_Cnty_SA_{y}_{m}.csv" for m in months]

    missing = [p for p in files if not p.exists()]
    if missing:
        raise FileNotFoundError(f"Missing service area file (example): {missing[0]}")

    sa = pd.concat([read_service_area(p) for p in files], ignore_index=True)
    sa["county_key"] = sa["fips"].where(sa["fips"].notna(), sa["ssa"])
    sa = sa[sa["county_key"].notna()].copy()

    return sa[["contractid", "county_key"]].drop_duplicates()

# ---------------- Inner merge + collapse ----------------
def inner_merge_service_area(plan_month: pd.DataFrame, sa_keyed: pd.DataFrame) -> pd.DataFrame:
    return plan_month.merge(sa_keyed, on=["contractid", "county_key"], how="inner")

def collapse_to_yearly_panel(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(["contractid", "planid", "county_key", "year", "month"], kind="mergesort")

    def summarize(g: pd.DataFrame) -> pd.Series:
        vals = g["enrollment"].dropna()
        return pd.Series({
            "avg_enrollment": vals.mean() if len(vals) else np.nan,
            "min_enrollment": vals.min() if len(vals) else np.nan,
            "max_enrollment": vals.max() if len(vals) else np.nan,
            "state": g["state"].iloc[-1],
            "county": g["county"].iloc[-1],
            "fips": g["fips"].iloc[-1],
            "ssa": g["ssa"].iloc[-1],
            "year": g["year"].iloc[-1],
        })

    return (
        df.groupby(["contractid", "planid", "county_key", "year"], dropna=False)
          .apply(summarize)
          .reset_index()
    )


In [13]:
plan_month = build_plan_month(2018)
print(plan_month.shape)
print(plan_month[["contractid","planid","state","county","fips","ssa","month"]].head())


(27710394, 20)
  contractid  planid state   county    fips      ssa  month
0      90091     NaN   NaN      NaN     NaN      NaN      1
1      E0654   801.0   NaN      NaN     NaN   2198.0      1
2      E0654   801.0   NaN      NaN     NaN  55830.0      1
3      E0654   801.0   NaN      NaN     NaN   2275.0      1
4      E0654   801.0    AL  Autauga  1001.0   1000.0      1


In [14]:
sa_keyed = build_service_area_keyed(2018)
print("Service area keys:", sa_keyed.shape)
print(sa_keyed.head())


Service area keys: (331547, 2)
  contractid  county_key
1      H0022       39023
2      H0022       39035
3      H0022       39051
4      H0022       39055
5      H0022       39057


In [15]:
plan_month_sa = inner_merge_service_area(plan_month, sa_keyed)

print("After service area merge:", plan_month_sa.shape)
print(plan_month_sa[["contractid","planid","state","county","fips","ssa","month"]].head())


After service area merge: (16094171, 20)
  contractid  planid state    county     fips      ssa  month
0      H0022     1.0    OH     Clark  39023.0  36110.0      1
1      H0022     1.0    OH  Cuyahoga  39035.0  36170.0      1
2      H0022     1.0    OH    Fulton  39051.0  36260.0      1
3      H0022     1.0    OH    Geauga  39055.0  36280.0      1
4      H0022     1.0    OH    Greene  39057.0  36290.0      1


In [14]:
# how many raw monthly rows
print("Raw monthly rows:", plan_month.shape[0])

# how many survive service-area restriction
print("After SA merge:", plan_month_sa.shape[0])

# how many unique plan–county combos
print(
    plan_month_sa[["contractid","planid","county_key"]]
    .drop_duplicates()
    .shape[0]
)


Raw monthly rows: 27710394
After SA merge: 16094171
1367240


In [20]:
sa_keyed = build_service_area_keyed(2018)

print("sa_keyed shape:", sa_keyed.shape)
print("sa_keyed columns:", list(sa_keyed.columns))
print(sa_keyed.head(10))


sa_keyed shape: (331547, 2)
sa_keyed columns: ['contractid', 'county_key']
   contractid  county_key
1       H0022       39023
2       H0022       39035
3       H0022       39051
4       H0022       39055
5       H0022       39057
6       H0022       39085
7       H0022       39093
8       H0022       39095
9       H0022       39103
10      H0022       39113


In [21]:
plan_month_sa = inner_merge_service_area(plan_month, sa_keyed)

print("plan_month shape (before SA merge):", plan_month.shape)
print("plan_month_sa shape (after SA merge):", plan_month_sa.shape)

# confirm the “blank county/state” rows are gone
print("Missing state after merge:", plan_month_sa["state"].isna().sum())
print("Missing county after merge:", plan_month_sa["county"].isna().sum())

print(plan_month_sa[["contractid","planid","state","county","fips","ssa","month","enrollment"]].head(10))


plan_month shape (before SA merge): (27710394, 20)
plan_month_sa shape (after SA merge): (16094171, 20)
Missing state after merge: 9009
Missing county after merge: 9009
  contractid  planid state      county     fips      ssa  month  enrollment
0      H0022     1.0    OH       Clark  39023.0  36110.0      1       558.0
1      H0022     1.0    OH    Cuyahoga  39035.0  36170.0      1      3596.0
2      H0022     1.0    OH      Fulton  39051.0  36260.0      1       107.0
3      H0022     1.0    OH      Geauga  39055.0  36280.0      1        80.0
4      H0022     1.0    OH      Greene  39057.0  36290.0      1       539.0
5      H0022     1.0    OH        Lake  39085.0  36440.0      1       278.0
6      H0022     1.0    OH      Lorain  39093.0  36480.0      1       559.0
7      H0022     1.0    OH       Lucas  39095.0  36490.0      1      2782.0
8      H0022     1.0    OH      Medina  39103.0  36530.0      1       192.0
9      H0022     1.0    OH  Montgomery  39113.0  36580.0      1      31

In [6]:
def inner_merge_service_area(plan_month, sa_keyed):
    return plan_month.merge(
        sa_keyed,
        on=["contractid", "county_key"],
        how="inner"
    )


In [8]:
plan_month = build_plan_month(2018)

print("plan_month exists:", type(plan_month))
print("plan_month shape:", plan_month.shape)
print(plan_month[["contractid","planid","state","county","fips","ssa","month"]].head())


NameError: name 'build_plan_month' is not defined

In [7]:
plan_month_sa = inner_merge_service_area(plan_month, sa_keyed)

print("After SA merge:", plan_month_sa.shape)


NameError: name 'plan_month' is not defined

In [2]:
plan_month_sa = plan_month_sa.dropna(subset=["state", "county"])
print("After dropping missing county labels:", plan_month_sa.shape)

NameError: name 'plan_month_sa' is not defined

In [1]:
final_plans = collapse_to_yearly_panel(plan_month_sa)

print("final_plans shape:", final_plans.shape)
print(final_plans.head(10))


NameError: name 'collapse_to_yearly_panel' is not defined

In [ ]:
out = OUT_DIR / "ma_plan_county_year_2018.csv"
final_plans.to_csv(out, index=False)
print("Saved:", out)


This code enables Python features and imports commonly used libraries for data analysis and file handling.

In [16]:
def build_plan_month(y: int) -> pd.DataFrame:
    months = monthlist_for_year(y)

    plan_month = pd.concat([load_month(m, y) for m in months], ignore_index=True)

    # make sure key columns exist + consistent types
    if "contractid" not in plan_month.columns:
        raise ValueError(f"'contractid' missing. Columns are: {list(plan_month.columns)}")

    # sort (good practice, not required)
    plan_month = plan_month.sort_values(
        ["contractid", "planid", "state", "county", "month"],
        kind="mergesort"
    )

    # county join key: prefer FIPS, else SSA
    plan_month["county_key"] = plan_month["fips"].where(
        plan_month["fips"].notna(),
        plan_month["ssa"]
    )

    return plan_month


In [17]:
CONTRACT_COLUMNS = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
]
ENROLL_COLUMNS = ["contractid", "planid", "ssa", "fips", "state", "county", "enrollment"]

def read_contract(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, skiprows=1, header=None, encoding="latin1", encoding_errors="replace", low_memory=False)
    df = df.iloc[:, :len(CONTRACT_COLUMNS)]
    df.columns = CONTRACT_COLUMNS
    return df

def read_enroll(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, skiprows=1, header=None, encoding="latin1", encoding_errors="replace",
                     na_values=["*"], low_memory=False)
    df = df.iloc[:, :len(ENROLL_COLUMNS)]
    df.columns = ENROLL_COLUMNS
    return df


In [18]:
def build_plan_month(y: int) -> pd.DataFrame:
    months = [f"{m:02d}" for m in range(1, 13)]
    plan_month = pd.concat([load_month(m, y) for m in months], ignore_index=True)

    # sanity check
    required = {"contractid","planid","state","county","fips","ssa","month"}
    missing = required - set(plan_month.columns)
    if missing:
        raise ValueError(f"Missing columns {missing}. Have: {list(plan_month.columns)}")

    plan_month["county_key"] = plan_month["fips"].where(plan_month["fips"].notna(), plan_month["ssa"])
    return plan_month


In [16]:
BASE_DIR = Path("Econ470/a0/work/ma/Extracted Data")


def monthlist_for_year(y: int) -> list[str]:
    months = range(7, 13) if y == 2006 else range(1, 13)
    return [f"{m:02d}" for m in months]


This code defines a base directory and a helper function for working with month values. Although my file path is still incorrect I need to make some changes to correct this. The function monthlist_for_year takes a year as input and returns a list of month strings in two-digit format ("01"–"12"). For the year 2006 only, it returns July through December ("07"–"12"), and for all other years it returns January through December, which works for us as we need data from 2018.

In [20]:
CONTRACT_COLUMNS = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
]
CONTRACT_DTYPES = {
    "contractid": "string",
    "planid": "float64",
    "org_type": "string",
    "plan_type": "string",
    "partd": "string",
    "snp": "string",
    "eghp": "string",
    "org_name": "string",
    "org_marketing_name": "string",
    "plan_name": "string",
    "parent_org": "string",
    "contract_date": "string",
}

ENROLL_COLUMNS = [
    "contractid", "planid", "ssa", "fips", "state", "county", "enrollment"
]
ENROLL_DTYPES = {
    "contractid": "string",
    "planid": "float64",
    "ssa": "float64",
    "fips": "float64",
    "state": "string",
    "county": "string",
    "enrollment": "float64",
}

This code defines metadata for the two types of datasets where one for contracts and one for enrollment by listing their expected column names and specifying the data type for each column. We need to ensure consistent column order and correct data types, which helps prevent parsing errors and makes downstream analysis more reliable.

In [11]:
def read_contract(path: Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=CONTRACT_COLUMNS,
        dtype=CONTRACT_DTYPES,
        low_memory=False,
    )


def read_enroll(path: Path) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=1,
        header=None,
        names=ENROLL_COLUMNS,
        dtype=ENROLL_DTYPES,
        na_values=["*"],
        low_memory=False,
    )



def fill_downup(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    df[cols] = df[cols].ffill().bfill()
    return df



In [12]:
def load_month(m: str, y: int) -> pd.DataFrame:
    c_path = BASE_DIR / f"CPSC_Contract_Info_{y}_{m}.csv"
    e_path = BASE_DIR / f"CPSC_Enrollment_Info_{y}_{m}.csv"

    contract_info = (
        read_contract(c_path)
        .drop_duplicates(subset=["contractid", "planid"], keep="first")
    )

    enroll_info = read_enroll(e_path)

    return (
        contract_info
        .merge(enroll_info, on=["contractid", "planid"], how="left")
        .assign(month=int(m), year=y)
    )


def build_plan_year(y: int) -> pd.DataFrame:
    monthlist = monthlist_for_year(y)

    plan_year = pd.concat(
        [load_month(m, y) for m in monthlist],
        ignore_index=True
    )

    plan_year = plan_year.sort_values(
        ["contractid", "planid", "state", "county", "month"],
        kind="mergesort"
    )

    plan_year = (
        plan_year
        .groupby(["state", "county"], dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["fips"]))
        .groupby(["contractid", "planid"], dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["plan_type", "partd", "snp", "eghp", "plan_name"]))
        .groupby("contractid", dropna=False, group_keys=False)
        .apply(lambda g: fill_downup(g, ["org_type", "org_name", "org_marketing_name", "parent_org"]))
        .reset_index(drop=True)
    )

    return plan_year


In [13]:
def collapse_to_yearly_panel(plan_year: pd.DataFrame) -> pd.DataFrame:
    plan_year = plan_year.sort_values(
        ["contractid", "planid", "fips", "year", "month"],
        kind="mergesort"
    )

    def summarize(g):
        enroll = g["enrollment"]
        nonmiss = enroll.notna().sum()
        vals = enroll.dropna()

        return pd.Series({
            "n_nonmiss": nonmiss,
            "avg_enrollment": vals.mean() if nonmiss > 0 else np.nan,
            "sd_enrollment": vals.std(ddof=1) if nonmiss > 1 else np.nan,
            "min_enrollment": vals.min() if nonmiss > 0 else np.nan,
            "max_enrollment": vals.max() if nonmiss > 0 else np.nan,
            "first_enrollment": vals.iloc[0] if nonmiss > 0 else np.nan,
            "last_enrollment": vals.iloc[-1] if nonmiss > 0 else np.nan,
            "state": g["state"].iloc[-1],
            "county": g["county"].iloc[-1],
            "org_type": g["org_type"].iloc[-1],
            "plan_type": g["plan_type"].iloc[-1],
            "partd": g["partd"].iloc[-1],
            "snp": g["snp"].iloc[-1],
            "eghp": g["eghp"].iloc[-1],
            "org_name": g["org_name"].iloc[-1],
            "org_marketing_name": g["org_marketing_name"].iloc[-1],
            "plan_name": g["plan_name"].iloc[-1],
            "parent_org": g["parent_org"].iloc[-1],
            "contract_date": g["contract_date"].iloc[-1],
            "year": g["year"].iloc[-1],
        })

    return (
        plan_year
        .groupby(["contractid", "planid", "fips", "year"], dropna=False)
        .apply(summarize)
        .reset_index()
    )



In [ ]:
if __name__ == "__main__":
    y = 2018
    plan_year = build_plan_year(y)
    final_plans = collapse_to_yearly_panel(plan_year)
     

This code takes raw plan data for 2018, processes it, and creates a summarized yearly panel of plans for the final combined data set

In [ ]:
import pandas as pd

# Count the number of plans for each plan_type
plan_counts = final_plans['plan_type'].value_counts().reset_index()

# Rename the columns for readability
plan_counts.columns = ['Plan Type', 'Number of Plans']

# Optional: sort by number of plans descending
plan_counts = plan_counts.sort_values(by='Number of Plans', ascending=False)

# Display the table
print(plan_counts)


this is the base of the code to create the table based on the plans 

In [ ]:
import pandas as pd

# Filter out special needs plans (SNP), employer group plans (EGHP), and 800-series plans
filtered_plans = final_plans[
    (final_plans['snp'] != 'Y') &                 # remove SNP plans
    (final_plans['eghp'] != 'Y') &               # remove EGHP plans
    (~final_plans['planid'].astype(str).str.startswith('8'))  # remove 800-series plan IDs
]

# Recompute the count of plans by plan_type
filtered_plan_counts = filtered_plans['plan_type'].value_counts().reset_index()
filtered_plan_counts.columns = ['Plan Type', 'Number of Plans']

# Optional: sort by number of plans descending
filtered_plan_counts = filtered_plan_counts.sort_values(by='Number of Plans', ascending=False)

# Display the updated table
print(filtered_plan_counts)

this is the base of the code to Remove all special needs plans

In [ ]:
import pandas as pd

# Merge filtered plans with service area to keep only approved counties
# Assuming service_area has columns: 'planid' and 'county'
approved_plans = filtered_plans.merge(
    service_area[['planid', 'county']],
    on=['planid', 'county'],
    how='inner'  # keeps only matches
)

# Compute average enrollment by plan_type
avg_enrollment = approved_plans.groupby('plan_type')['enrollment'].mean().reset_index()

# Rename columns for clarity
avg_enrollment.columns = ['Plan Type', 'Average Enrollment']

# Optional: sort descending by average enrollment
avg_enrollment = avg_enrollment.sort_values(by='Average Enrollment', ascending=False)

# Display the table
print(avg_enrollment)
